`GB` BigData / [Олег Гладкий](https://gb.ru/users/3837199) // домашнее задание

`262698` __Методы сбора и обработки данных из сети Интернет__:  `03`. Парсинг даных: HTML, __Beautiful-Soap__


## Задание 1. Beautiful-Soap. hh.ru

Собрать информацию о вакансиях на вводимую должность с сайтов `hh.ru` и/или `Superjob` и/или `работа.ру`. Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе минимум:
* Наименование вакансии.
* Предлагаемую зарплату (дополнительно: разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
* Ссылку на саму вакансию.
* Сайт, откуда собрана вакансия.


По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий со всех сайтов. Общий результат можно вывести с помощью `dataFrame` через `pandas`, сохранить в `json`, либо `csv`.

Минимум — один сайт, МАКСИМУМ — все два...

In [1]:
from bs4 import BeautifulSoup as bs
import requests

import time
import datetime
import pandas as pd
import numpy as np
import json
import math

import platform  # информация о системе 

from pprint import pprint 
# import re
# from lxml import html  # xpath

### Функции

__Money__, фунукция `get_money()` возвращает зарплатный словарь, полученный разбором строки от сайта:
* минимальное значение
* максимальное значение
* валюту зарплаты

Соглашения, для соответствия указаниям сайта и значениям словаря на примере указания сайта:
* от 1000 р.
    * `pay['Maney_min']` = 1000
    * `pay['Maney_max']` = 0
* от 1000 до 3000 р.
    * `pay['Maney_min']` = 1000
    * `pay['Maney_max']` = 3000 
* до 3000 р.
    * `pay['Maney_min']` = 0
    * `pay['Maney_max']` = 3000
* 1000 р.
    * `pay['Maney_min']` = 1000
    * `pay['Maney_max']` = 1000



In [2]:
def get_maney(money_range_str:str="") ->dict:
    
    ''' Выделим цифровые значения зарплаты из строки '''
    
    pay = {'min': 0, 'max': 0, 'curr': ''}
    
    if not money_range_str:
        return pay
    
    # Специальные пробельные символы необходимо "схлопнуть"
    
    special_space_symbols = {
        "NARROW_NO_BREAK_SPACE": "\u202f", 
        "NO_BREAK_SPACE": "\u00a0",
        "TAB":"\t",
    }

    for key, space_code in special_space_symbols.items():
        money_range_str = money_range_str.replace(space_code, "")

    # Анализируем: 
    
    money_words = money_range_str.split(" ")  # Раделим строку на слова
    i = 0
    while money_words:
        i += 1
        if i  > 10:
            print("Выход из бесконечного цикла")
            break
      
        if not money_words[-1].isdigit():
            pay['curr'] = money_words[-1]
            money_words.pop(-1)
            continue

        if money_words[0].upper() == "от".upper() or \
           money_words[0].upper() == "from".upper():
            pay['min'] = int(money_words[1])
            money_words.pop(1)
            money_words.pop(0)
            continue

        if money_words[0].upper() == "до".upper():
            pay['max'] = int(money_words[1])
            money_words.pop(1)
            money_words.pop(0)
            continue
            
        if money_words[0] == "-" or money_words[0] == "–" or money_words[0] == "—" or money_words[0] == "--":
            pay['max'] = int(money_words[1])
            money_words.pop(1)
            money_words.pop(0)
            continue
            
        if money_words[0].isdigit():
            pay['min'] = int(money_words[0])
            money_words.pop(0)
            continue
        else:
            money_words.pop(0)
            continue
            
    else:
        if not pay['min']:
            pay['min'] = pay['max']
        if not pay['max']:
            pass
            # pay['max'] = pay['min']
        pass
          
    return pay

__Soup__, фунукция `get_soup()`, возвращает струтуру `bs4.BeautifulSoup` 

In [3]:
def get_soup(url:str=None, headers:str=None, params:str=None) -> bs("", 'html.parser'):
    
    if not url: return None  # проверка...
    
    response = requests.get(url=url, headers=headers, params=params)
    # print(response.url)

    if not response.ok:
        print(response.status_code, response.url)
        return None
    
    if response.headers['Content-Type'] != 'text/html; charset=utf-8':
        print(response.headers['Content-Type'])
        return None
    
    soup = bs(response.text, 'html.parser')
    
    return soup 

__Вакансии__: функция `get_vacancies_hh_parser` парсит вакансии с сайта `hh.ru` из полученной струтуры `soup`

In [4]:
def get_vacancies_hh_parser(soup="") -> list: 
    
    ''' запрос вакансий с сайта hh.ru '''
    
    if not soup: return None
    
    cards = soup.find_all(name='div', attrs={'class': ['serp-item'], 'data-qa': True})
    
    if len(cards) == 0:
        return None
    
    vacancies = list()
    
    for i, card in enumerate(cards, start=1): 
        name = card.find(name='a', attrs={'class': 'serp-item__title'}).text
        link = card.find(name='a', attrs={'class': 'serp-item__title'}).get('href').split('?')[0]
       
        try:
            comp = card.find(name='a', attrs={'class': ['bloko-link', 'bloko-link_kind-tertiary'], 'data-qa': 'vacancy-serp__vacancy-employer'}).text
            
        except:
            comp = "__AttributeError__"
            print(f"AttributeError, item {i}")
            #comt = card.find(name='div', attrs={'class': 'vacancy-serp-item__meta-info-company'}.text)
            
        maney = card.find(name='span', attrs={'data-qa': 'vacancy-serp__vacancy-compensation'})
        maney = maney.get_text(strip=False) if maney else None 
        maney = get_maney(maney) # разбор строки в список значений словаря...
        
        
        vacancy = {'Name': name, 'Company': comp, 'Link': link, 'Date': request_date, 'Source': url, \
                   'Maney_min': maney['min'], 'Maney_max': maney['max'], 'Maney_curr': maney['curr']}
        
        vacancies.append(vacancy)  # type: list
        
    return vacancies

__Страницы__: функция `get_pages()` парсит количество страниц по данному запросу из полученной струтуры `soup`.

In [1]:
def get_pages(soup=None, item_on_page=None) ->int:
    
    '''Определяем количество страниц в ответе от сервера'''
    
    if not soup or not item_on_page: 
        print("get_pages() -- пустой вызов")
        return None
    
    items = soup.find(name='h1', attrs={'class': ['bloko-header-section-3'], 'data-qa': ['bloko-header-3']}).text
    items = get_maney(items)['min']

    pages = int(math.ceil(items/item_on_page))
    
    return pages

### Параметры доступа к сайту

Адрес сайта, параметры и заголовки. Агент, а так же количество позиций на страничке.

In [3]:
if platform.system() == 'Windows':
    win_ver = platform.release()
    if win_ver == '7': # Windows 7 User-Agent
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}
    elif win_ver == '10':  # Windows 10 User-Agent        
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',}
else:
    print("ВНИМАНИЕ! ЮЗЕР-АГЕНТ НЕ СООТВЕТСТВУЕТ СИСТЕМЕ")
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}

item_on_page = 20
url = 'https://hh.ru/search/vacancy/'
params = {
    'area':1,
    'clusters':'true',
    'enable_snippets':'true',
    'items_on_page':item_on_page,
    'ored_clusters':'true',
    'search_field':'description',
    'text':'python',
    'order_by':'publication_time',
    'hhtmFrom':'vacancy_search_list',
}

NameError: name 'platform' is not defined

## Поехали!

Запрашиваем странички и парсим их... С небольшим таймаутом, чтобы сервер «не подумал», что мы его парсим.

In [14]:
%%time
sleep_interval = 2  # задержка между запросами
request_date = str(datetime.date.today())
request_time = str(datetime.datetime.now().strftime("%H:%M"))

vacancies_dict = dict()
vacancies = list()
soup = ""
pages = 1
page = 0

while page < pages: 

    params['page'] = page
    soup = get_soup(url=url, headers=headers, params=params)

    if soup:
        vacancies_part = get_vacancies_hh_parser(soup)
        try: 
            if len(vacancies_part) > 0 and vacancies_part != None:
                vacancies.extend(vacancies_part)
                # vacancies = vacancies + vacancies_part
        except:
            print(f"Страница {page+1} не обработана, ошибка.")
                
        time.sleep(sleep_interval)
    else:
        print(f"Страницы выборки закончились, всего получено и обработано {page} страниц.")
        break

    if page < 1:
        pages = get_pages(soup=soup, item_on_page=item_on_page)
        pass
        
    print(f"Страница {page+1} обработана успешно, всего страниц {pages}")
    page += 1

else:
    print(f"Получено {len(vacancies)} вакансий")    



Страница 1 обработана успешно, всего страниц 223
Страница 2 обработана успешно, всего страниц 223
Страница 3 обработана успешно, всего страниц 223
Страница 4 обработана успешно, всего страниц 223
Страница 5 обработана успешно, всего страниц 223
Страница 6 обработана успешно, всего страниц 223
Страница 7 обработана успешно, всего страниц 223
Страница 8 обработана успешно, всего страниц 223
Страница 9 обработана успешно, всего страниц 223
Страница 10 обработана успешно, всего страниц 223
Страница 11 обработана успешно, всего страниц 223
Страница 12 обработана успешно, всего страниц 223
Страница 13 обработана успешно, всего страниц 223
Страница 14 обработана успешно, всего страниц 223
Страница 15 обработана успешно, всего страниц 223
Страница 16 обработана успешно, всего страниц 223
Страница 17 обработана успешно, всего страниц 223
Страница 18 обработана успешно, всего страниц 223
Страница 19 обработана успешно, всего страниц 223
Страница 20 обработана успешно, всего страниц 223
Страница 

__Запись__ данных на диск. 
Записываем все полученные данные в файл `03_hw_HTML_Beautiful-Soap_VACANCIES` на диск в `json`-формате.

In [8]:
# Записываем результат на диск (пишем сразу всё, а не частично)

with open('03_hw_HTML_Beautiful-Soap_VACANCIES.json', 'w') as out_f:
    json.dump(vacancies, out_f)

__Вид__ в формате таблицы `DataFrame`

In [13]:
vacancies_DF = pd.DataFrame()
vacancies_DF = vacancies_DF.append(vacancies, ignore_index=True)
print(f"Размер таблицы DataFrame: {vacancies_DF.shape}, Строк: {len(vacancies_DF)}")
vacancies_DF

Размер таблицы DataFrame: (2000, 8), Строк: 2000


,Name,Company,Link,Date,Source,Maney_min,Maney_max,Maney_curr
0,Специалист по информационной безопасности / Де...,ООО Центр корпоративных решений,https://adsrv.hh.ru/click,2022-10-10,https://hh.ru/search/vacancy/,0,0,
1,Старший продуктовый аналитик в команду «Поиск ...,Профи (profi.ru),https://nahabino.hh.ru/vacancy/67773818,2022-10-10,https://hh.ru/search/vacancy/,250000,350000,руб.
2,Продуктовый аналитик в команду «Поиск заказов»,Профи (profi.ru),https://nahabino.hh.ru/vacancy/70479973,2022-10-10,https://hh.ru/search/vacancy/,0,0,
3,Продуктовый аналитик в направление «Работодате...,ООО HeadHunter::Product Management,https://nahabino.hh.ru/vacancy/69072132,2022-10-10,https://hh.ru/search/vacancy/,0,0,
4,Data Quality Engineer,ООО HeadHunter:: IT,https://nahabino.hh.ru/vacancy/67932769,2022-10-10,https://hh.ru/search/vacancy/,0,0,
...,...,...,...,...,...,...,...,...
1995,Руководитель управления агрегированных рисков,БКС Бизнес и процессы,https://nahabino.hh.ru/vacancy/68439192,2022-10-10,https://hh.ru/search/vacancy/,0,0,
1996,Full-stack developer,Telecom Биржа,https://nahabino.hh.ru/vacancy/68169304,2022-10-10,https://hh.ru/search/vacancy/,0,0,
1997,Linux administrator/Support engineer 2L,SberTech,https://nahabino.hh.ru/vacancy/69629710,2022-10-10,https://hh.ru/search/vacancy/,150000,250000,руб.
1998,Эксперт платформы виртуализации серверов (Инже...,Сбер. IT,https://nahabino.hh.ru/vacancy/55566252,2022-10-10,https://hh.ru/search/vacancy/,0,0,


<!--  -->

<!--  -->

<!--  -->

__P.S.__



* Ошибку получения атрибута `text` для названия компании `comp` победить не удалось — отложил на будущее. Но в том месте, где это происходило встречались какие-то комментарии... 
* Почему-то более 100 страниц сайт `hh.ru` не отдаёт...

<!--  -->

ZIP of code...

In [10]:
%%time
# инициализируем выходную таблицу
# vacancies = pd.DataFrame( \
#     columns=['Name', 'Company', 'Link', 'Date', 'Source', 'Maney_min', 'Maney_max', 'Maney_curr'], \
#     index=[])

Wall time: 0 ns
